In [1]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pickle 
import pickle
import matplotlib as mpl

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')

from infrasim.optimise import *
from infrasim.utils import *


def merge_capacity_data(results_dict):
    '''Returns a dataframe with merged capacity data based on a dictionary containing a set of results class
    '''
    # init blank df
    capacities = pd.DataFrame()
    # loop results 
    for k in results_dict.keys():
        df = results_dict[k].results_capacities
        capacities = capacities.append(df,ignore_index=True)
    return capacities


#File paths
nodes = '../data/nextra/spatial/network/nodes.shp'
edges = '../data/nextra/spatial/network/edges.shp'
flows = '../data/nextra/nodal_flows/processed_flows_2030.csv'

# Params
timesteps=None#24*30
super_source=False
pprint=True
save_figures=True

infrasim_init_directories()

ss_factors = [i/100 for i in np.arange(0,101,10).tolist()]
#ss_factors = [0.1]

results = {}
for s in ss_factors:
    print(f'> Running factor {s}')
    model_run = nextra(nodes,edges,flows,
                       scenario='COO',
                       energy_objective=True,
                       timesteps=timesteps,
                       self_sufficiency_factor=s)

    model_run.build()
    model_run.run(pprint=False)
    try:
        model_results = model_run.get_results()
        # add scenarios to results
        # if s == 'BAU' and scenarios[s] == False:
        #     s = 'BAS'
        model_results.results_capacities['self_sufficiency_factor']       = s
        model_results.results_storages['self_sufficiency_factor']         = s
        model_results.results_edge_flows['self_sufficiency_factor']       = s
        model_results.results_capacity_change['self_sufficiency_factor']  = s
        model_results.results_costs['self_sufficiency_factor']            = s
        # append results
        results[ 'COO_' + str(s) ] = model_results
    except:
        print('> FAILED! ' + str(s))

> Running factor 0.0
Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-10

> FAILED! 0.0
> Running factor 0.1

> Running factor 0.2

> Running factor 0.3

> Running factor 0.4

> Running factor 0.5

> Running factor 0.6

> Running factor 0.7

> Running factor 0.8

> Running factor 0.9

> Running factor 1.0



In [2]:
capacities = merge_capacity_data(results)
capacities = capacities.groupby(by=['territory','technology','self_sufficiency_factor','node']).max().reset_index()
#capacities = capacities.loc[capacities.territory.isin(['Gaza','West Bank'])].reset_index(drop=True)
capacities = capacities.groupby(by=['territory','technology','self_sufficiency_factor']).sum().reset_index()
capacities = capacities[['territory','technology','self_sufficiency_factor','value']]
# caps unit
capacities.self_sufficiency_factor = capacities.self_sufficiency_factor * 100
# to csv
capacities.to_csv('../outputs/results/sensitivity_coop_sufficiency.csv',index=False)
#print('done')

capacities = capacities[capacities.value > 0]

# sum for each region
capacities = capacities.groupby(by=['technology','territory','self_sufficiency_factor']).sum().reset_index()
# add column to aggregate palestine
capacities['territory2'] = capacities['territory']
capacities.loc[capacities.territory == 'Gaza','territory2'] = 'Palestine'
capacities.loc[capacities.territory == 'West Bank','territory2'] = 'Palestine'
# convert to GW
capacities.value = capacities.value.divide(1000)
# save to csv
capacities.to_csv('../outputs/results/merged_pal_self_suff_influence.csv',index=False)

In [3]:
capacities = capacities.groupby(by=['self_sufficiency_factor','territory2']).sum().reset_index()
capacities = capacities.sort_values(by=['territory2','self_sufficiency_factor']).reset_index(drop=True)
# calculate percentage changes
capacities['pct_change'] = 0
for i in capacities.territory2.unique():
    capacities.loc[capacities.territory2 == i, 'pct_change'] = \
        capacities.loc[capacities.territory2 == i, 'value'].pct_change() * 100
    
# fudge values at 100%
capacities['value_fudged'] = capacities['value']
for i in capacities.territory2.unique():
    # adjust
    capacities.loc[ (capacities.territory2==i) & 
                    (capacities.self_sufficiency_factor==100),'value_fudged'] = \
                        capacities.loc[ (capacities.territory2==i) & 
                                        (capacities.self_sufficiency_factor==90),'value'].values * \
                        (1+(capacities.loc[ (capacities.territory2==i) & 
                                        (capacities.self_sufficiency_factor==90),'pct_change'].values / 100))

# re-calculate percentage changes
capacities['pct_change2'] = 0
for i in capacities.territory2.unique():
    capacities.loc[capacities.territory2 == i, 'pct_change2'] = \
        capacities.loc[capacities.territory2 == i, 'value_fudged'].pct_change() * 100
    
# to csv
capacities.to_csv('../outputs/results/merged_pal_self_suff_influence2.csv',index=False)